In [746]:
import datetime
import pandas as pd
import numpy as np
import os
import sys
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
import json

loc = os.getcwd()
sys.path.append(os.path.abspath('nytimesarticle-0.1.0'))
from get_NYT import get_NYT

from get_fx import get_fx
with open('keys.json') as keys:
    creds = json.load(keys)

In [728]:
if __name__ == '__main__':
    
    today = datetime.date.today()
    today_dt = str(today.year)+str(today.month).zfill(2)+str(today.day).zfill(2)
    brexit_dt = datetime.date(2016,6,22)
    start = str(brexit_dt.year)+str(brexit_dt.month).zfill(2)+str(brexit_dt.day).zfill(2)
    hist = today - brexit_dt

    '''Load price change file
    '''
    df_chg = pd.read_csv('GBPUSD_CHNG.csv',header=None,names=['date','change'])
    df_chg = df_chg.set_index(pd.DatetimeIndex(df_chg['date']))

    '''Grab new FX prices and calculate changes
    '''
    max_dt = max(pd.to_datetime(df_chg['date']))
    begin = max_dt.date() + datetime.timedelta(1)
    if max_dt.date() < today:
        fx_price_gap = get_fx(creds['ACCESS_KEY'], creds['url'], begin, today)
        dt_index = pd.date_range(max_dt,periods=(pd.to_datetime(today)-max_dt).days+1,freq='D')[::-1][:-1]
        dt_gap = [str(d.month)+'/'+str(d.day)+'/'+str(d.year) for d in dt_index]
        dt_gap = pd.DataFrame(dt_gap)
        fx_prices_more = pd.DataFrame(fx_price_gap)
        fx_px_delta = pd.concat([dt_gap,fx_prices_more], axis=1)
        fx_px_delta.columns = ['date','price']
        fx_price_hist = pd.read_csv('GBPUSD.csv',names=['date','price'],header=0)
        fx_prices = pd.concat([fx_px_delta,fx_price_hist],axis=0)
        fx_prices.to_csv('GBPUSD.csv',index=False)
        fx_prices = pd.read_csv('GBPUSD.csv',names=['date','price'],header=0)

        '''Calculate 1-day price change directions
        '''
        df_move = np.array(fx_prices.iloc[:-1,1]) - np.array(fx_prices.iloc[1:,1])
        df_signal = np.sign(df_move)
        df_chg = pd.Series(df_signal,index=fx_prices.iloc[:-1,0])
        df_chg.to_csv('GBPUSD_CHNG.csv')

In [542]:
    '''Backfill NYT articles'''
    df_NYT = pd.read_csv('NYTimes_20160623-20161202.csv',header=0,names=['dates','content'])
    df_NYT.set_index('dates',inplace=True)
    max_NYT_dt = max(pd.to_datetime(df_NYT.index))
    if max_NYT_dt.date() < today:
        begin_NYT = max_NYT_dt.date() + datetime.timedelta(1)
        begin_dt = str(begin_NYT.year)+str(begin_NYT.month).zfill(2)+str(begin_NYT.day).zfill(2)
        dts_NYT_fill, corpus_NYT_fill = get_NYT(begin_dt, today_dt)
        datetimes = [pd.to_datetime(dt) for dt in dts_NYT_fill]
        max_dt = max(datetimes)
        curr_articles = sum(np.array(datetimes) == max_dt)
        print "number of articles today: {}".format(curr_articles)
        #dt_NYT_gap = [d.strftime('%m/%d/%y') for d in dts_NYT_fill]
        dts_fill_NYT = [str(dt.month)+'/'+str(dt.day)+'/'+str(dt.year-2000) for dt in dts_NYT_fill]
        dt_NYT_gap = pd.DataFrame(dts_fill_NYT)
        df_NYT_corpus = pd.DataFrame(corpus_NYT_fill)
        pd_NYT_fill = pd.concat([dt_NYT_gap,df_NYT_corpus], axis=1)
        pd_NYT_fill.columns=['date','content']
        pd_NYT_fill.set_index('date',inplace=True)
        pd_NYT_fill

5
number of articles today: 1


,content
date,
2016-12-05,ROME Italians voted in record numbers on S...
2016-12-05,Global markets were not lacking in precarious ...
2016-12-05,LONDON In one of the most significant cons...
2016-12-05,Prime Minister Matteo Renzi offered his re...
2016-12-08,BURNASTON England On s brightly lit as...


In [550]:
    if dts_NYT_fill: 
        max_dt = dts_NYT_fill[-1]
        end_str = [int(d) for d in max_dt.split('-')]
        end_dt = datetime.date(end_str[0],end_str[1],end_str[2])
        ind_test = str(end_dt.month)+'/'+str(end_dt.day)+'/'+str(end_dt.year-2000)

2016-12-08


In [591]:
dt_comp = [dt.split('-') for dt in dts_NYT_fill]
#dts_NYT_fill = [dt[1]+'/'+str(int(dt[2]))+'/'+str(int(dt[0])) for dt in dt_comp]
dt_temp = [dt[0].split('/') for dt in dt_comp]
dts_NYT_fill = [dt[0]+'/'+str(int(dt[1]))+'/'+str(int(dt[2])) for dt in dt_temp]

In [612]:
    #dts_fill_NYT = [dt.split('-') for dt in dts_NYT_fill]
    dt_NYT_gap = pd.DataFrame(dts_NYT_fill)
    df_NYT_corpus = pd.DataFrame(corpus_NYT_fill)
    pd_NYT_fill = pd.concat([dt_NYT_gap,df_NYT_corpus], axis=1)
    pd_NYT_fill.reset_index(inplace=True, drop=True)
    pd_NYT_fill.columns=['dates','content']
    df_NYT_update = df_NYT[:51].append(pd_NYT_fill,ignore_index=True)
    df_NYT_update.to_csv('NYTimes_20160623-20161208.csv')

In [616]:
df_NYT.set_index('dates')

,content
dates,
6/23/16,Twilio which makes software that helps compan...
6/23/16,LONDON Ask for his opinion and David Howe...
6/23/16,LONDON Politicians campaigning for and aga...
6/23/16,LONDON Battle lines have been sharply draw...
6/23/16,LONDON If wakes up on Friday morning to t...
6/27/16,LONDON British cultural organizations and ...
6/27/16,American officials struggling to reimagine the...
6/27/16,LONDON Luke Hickmore had an aperitif Thurs...
6/27/16,CANNES France The invasion has begun In t...


In [617]:
    df_NYT = pd.read_csv('NYTimes_20160623-20161208.csv',header=0,names=['dates','content'])
    df_NYT.set_index('dates',inplace=True)
    corpus = df_NYT.loc[:ind_test]['content']
    mask = corpus.index.isin([ind_test])
    corpus_train = corpus.loc[~mask]
    X_train = count_vec.fit_transform(corpus_train)

In [680]:
'''
max_dt = pd.to_datetime('12/8/2016')
indx_test = pd.date_range(max_dt,periods=(pd.to_datetime(today)-max_dt).days+1,freq='D')[::-1][:-1]
[str(d.month)+'/'+str(d.day)+'/'+str(d.year) for d in indx_test]
'''
#dt_temp = [dt.split('/') for dt in df_chg.index]
#dt_col = [str(dt[0])+'/'+str(int(dt[1]))+'/'+str(dt[2]) for dt in dt_temp]
#df_chg.reset_index(inplace=True, drop=True)
#df_chg.values
#df_chg = pd.concat([df_chg,pd.Series(dt_col)],axis=1,names=['change','dates'])
df_chg.rename(columns ={0:'change',1:'dates'},inplace=True)
#df_chg.drop(axis=1,level=2,inplace=True)
df_chg = pd.concat([df_chg.ix[:,1],df_chg.ix[:,0]], axis=1)

In [685]:
df_chg.set_index('dates',inplace=True)
df_chg

,change
dates,
12/10/16,0
12/9/16,-1
12/8/16,-1
12/7/16,-1
12/6/16,-1
12/5/16,1
12/4/16,-1
12/3/16,0
12/2/16,1


In [698]:
    Y = []
    for dt in corpus.index:
        '''Remove leading zeros from 12/05/16'''
        Y.append(df_chg.loc[dt]['change'])
    df_px_article = pd.DataFrame(Y,index=pd.DatetimeIndex(corpus.index),columns=['PriceChg'])
    df_px_article

,PriceChg
2016-06-23,1
2016-06-23,1
2016-06-23,1
2016-06-23,1
2016-06-23,1
2016-06-27,-1
2016-06-27,-1
2016-06-27,-1
2016-06-27,-1
2016-06-27,-1


In [701]:
df_NYT.set_index('dates', inplace=True)

In [711]:
len(corpus_test)

1

In [ ]:
    count_vec = CountVectorizer(min_df=1,stop_words='english',lowercase=True)
    tfidf_vec = TfidfVectorizer(sublinear_tf=True, max_df=0.5,stop_words='english')

In [721]:
    corpus_test = df_NYT.loc[ind_test]['content']
    if isinstance(corpus_test, str):
        corpus_test = [corpus_test]
    else: 
        corpus_test = [d for d in corpus_test]
    corpus_train = df_NYT[:-len(corpus_test)]['content']
    X_train = count_vec.fit_transform(corpus_train)
    X_test = count_vec.transform(corpus_test)
    Y_train = Y[:-len(corpus_test)]
    Y_test = Y[-len(corpus_test):]

In [722]:
print len(Y_train)
print X_train.shape

55
(55, 7200)


In [723]:
print len(Y_test)
print X_test.shape

1
(1, 7200)


In [724]:
    clf = MultinomialNB(alpha=1.0, fit_prior=True, class_prior=None)
    clf.fit(X_train, Y_train)
    print clf.predict(X_test)

[ 1.]


In [720]:
    X_tfidf_train = tfidf_vec.fit_transform(corpus_train)
    X_tfidf_test = tfidf_vec.transform(corpus_test)
    clf.fit(X_tfidf_train, Y_train)
    print clf.predict(X_tfidf_test)

[-1.]
